In [17]:
import tomopy
import dxchange
import matplotlib.pyplot as plt
import time
import binascii
import tempfile

## Spark lightsource-(Sirt,Gridrec) Total time to Completion 

In [50]:
%%time 
def lightsource_reconstruction_spark(message, algo='gridrec'):
    
    msg_bin = binascii.unhexlify(message)
    tf = tempfile.NamedTemporaryFile(delete=True)
    tf.write(msg_bin)
    tf.flush()
    
    start = 0
    end = 2
    
    proj, flat, dark = dxchange.read_aps_32id(tf.name, sino=(start,end))
    theta = tomopy.angles(proj.shape[0])



    if (theta is None):
        theta = tomopy.angles(proj.shape[0])
    else:
        pass
    
    proj = tomopy.normalize(proj, flat, dark)
    rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
    proj = tomopy.minus_log(proj)
    recon = tomopy.recon(proj, theta, center=rot_center, algorithm=algo)
    
    tf.close()
    
    return 0


## create message
data_file = 'tooth.h5'
with open(data_file,'r') as f:
    data = f.read()

data_enc = binascii.hexlify(data).encode('utf-8')

data_list = []
for i in range(1):
    data_list.append(data_enc)
    
data_list_rdd = sc.parallelize(data_list,len(data_list))

tts = time.time()
res = data_list_rdd.map(lightsource_reconstruction_spark).collect()
tte = time.time()
print res
print 'gridrec TTC is: ', (tte-tts)

    
    

[0]
gridrec TTC is:  5.4723470211
CPU times: user 14.8 ms, sys: 7.54 ms, total: 22.4 ms
Wall time: 5.49 s


In [54]:
%%time 
def lightsource_reconstruction_spark(message, algo='sirt'):
    
    msg_bin = binascii.unhexlify(message)
    tf = tempfile.NamedTemporaryFile(delete=True)
    tf.write(msg_bin)
    tf.flush()
    
    start = 0
    end = 2
    
    proj, flat, dark = dxchange.read_aps_32id(tf.name, sino=(start,end))
    theta = tomopy.angles(proj.shape[0])



    if (theta is None):
        theta = tomopy.angles(proj.shape[0])
    else:
        pass
    
    proj = tomopy.normalize(proj, flat, dark)
    rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
    proj = tomopy.minus_log(proj)
    recon = tomopy.recon(proj, theta, center=rot_center, algorithm=algo)
    
    tf.close()
    
    return 0


## create message
data_file = 'tooth.h5'
with open(data_file,'r') as f:
    data = f.read()

data_enc = binascii.hexlify(data).encode('utf-8')

data_list = []
for i in range(1):
    data_list.append(data_enc)
    
data_list_rdd = sc.parallelize(data_list,len(data_list))

tts = time.time()
res = data_list_rdd.map(lightsource_reconstruction_spark).collect()
tte = time.time()
print res
print 'sirt TTC is: ', (tte-tts)

[0]
sirt TTC is:  7.20622396469
CPU times: user 16 ms, sys: 9.07 ms, total: 25.1 ms
Wall time: 7.23 s


## Lightsource-(Sirt,Gridrec) Total time to completion vanilla 

In [38]:
def lightsource_reconstruction(message,algorithm='gridrec'):

    tt = time.time()

    msg_bin = binascii.unhexlify(message)
    tf = tempfile.NamedTemporaryFile(delete=True,suffix='.h5')
    tf.write(msg_bin)
    tf.flush()

    tt = time.time() - tt


    start_time = time.time()


    start = 0
    end = 2

    proj, flat, dark= dxchange.read_aps_32id(tf.name, sino=(start, end))
    theta = tomopy.angles(proj.shape[0])



    if (theta is None):
        theta = tomopy.angles(proj.shape[0])
    else:
        pass

    proj = tomopy.normalize(proj, flat, dark)
    rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
    proj = tomopy.minus_log(proj)
    recon = tomopy.recon(proj, theta, center=rot_center, algorithm=algorithm)

    end_time = time.time()
    ttc = end_time - start_time

    #print 'Total time to completion of ' + algorithm  + '  algorithm is: ',  ttc
    #print 'Total time to decode is: ' + str(tt)

    tf.close()

    return 0



## create message:
start = time.time()
data_file = 'tooth.h5'
with open(data_file, "r") as f:
    data = f.read()
data_enc = binascii.hexlify(data).encode('utf-8')
end_data_generation = time.time()

aa = time.time()
for i in range(1):
    lightsource_reconstruction(data_enc,'gridrec')

print 'Gridrec ttc is:', str(time.time() - aa)


aa = time.time()
for i in range(1):
    lightsource_reconstruction(data_enc,'sirt')

print 'Sirt ttc is:', str(time.time() - aa)




Gridrec ttc is: 2.27679395676
Sirt ttc is: 3.95583701134


In [56]:
for i in range(10):
    aa = time.time()
    for i in range(1):
        lightsource_reconstruction(data_enc,'sirt')

    print 'Sirt ttc is:', str(time.time() - aa)


Sirt ttc is: 3.85913586617
Sirt ttc is: 3.77250313759
Sirt ttc is: 3.80923914909
Sirt ttc is: 3.54436302185
Sirt ttc is: 3.53715085983
Sirt ttc is: 3.52626299858
Sirt ttc is: 3.4988720417
Sirt ttc is: 3.61956191063
Sirt ttc is: 3.84473419189
Sirt ttc is: 3.8027920723
